# Smart Buildings Simulation Visualizer

- **Columns**: Weather scenarios (cold_winter → tropical)
- **Rows**: Policies (all_off, all_on, bang_bang)

In [ ]:
import os
import json
from pathlib import Path
from IPython.display import display, HTML, Image
import pandas as pd

RESULTS_DIR = Path("distributed_results")
WEATHERS = ["cold_winter", "mild_winter", "temperate", "mild_summer", "hot_summer", "tropical"]
POLICIES = ["all_off", "all_on", "bang_bang"]

WEATHER_NAMES = {
    "cold_winter": "Cold Winter", "mild_winter": "Mild Winter", "temperate": "Temperate",
    "mild_summer": "Mild Summer", "hot_summer": "Hot Summer", "tropical": "Tropical"
}
POLICY_NAMES = {"all_off": "All Off", "all_on": "All On", "bang_bang": "Bang-Bang"}

In [ ]:
def list_buildings():
    if not RESULTS_DIR.exists():
        return []
    return sorted([d.name for d in RESULTS_DIR.iterdir() if d.is_dir()])

def count_complete(building):
    count = 0
    for w in WEATHERS:
        for p in POLICIES:
            if (RESULTS_DIR / building / w / p / "summary.json").exists():
                count += 1
    return count

def load_summaries(building):
    summaries = []
    for weather in WEATHERS:
        for policy in POLICIES:
            path = RESULTS_DIR / building / weather / policy / "summary.json"
            if path.exists():
                with open(path) as f:
                    data = json.load(f)
                    data['weather'] = weather
                    data['policy'] = policy
                    summaries.append(data)
    return pd.DataFrame(summaries)

print("Available buildings:")
for b in list_buildings():
    print(f"  {b}: {count_complete(b)}/18 complete")

In [ ]:
# SELECT YOUR BUILDING HERE
BUILDING = "building_0002"

## Rewards Heatmap

In [ ]:
df = load_summaries(BUILDING)
if len(df) > 0:
    pivot = df.pivot(index='policy', columns='weather', values='total_reward')
    pivot = pivot.reindex(columns=[w for w in WEATHERS if w in pivot.columns])
    pivot = pivot.reindex([p for p in POLICIES if p in pivot.index])
    print(f"Total Rewards for {BUILDING}:")
    display(pivot.style.format("{:.1f}").background_gradient(cmap='RdYlGn', axis=None))
else:
    print("No completed scenarios yet.")

## GIF Grid

In [ ]:
def create_gif_grid(building, max_width=180):
    """Create HTML grid of GIFs using relative file paths."""
    html = f'''
    <style>
        .gif-grid {{ border-collapse: collapse; margin: 20px 0; }}
        .gif-grid th, .gif-grid td {{ border: 1px solid #ddd; padding: 8px; text-align: center; vertical-align: middle; }}
        .gif-grid th {{ background-color: #4a90d9; color: white; font-weight: bold; }}
        .gif-grid .policy-header {{ background-color: #5ba55b; color: white; font-weight: bold; }}
        .gif-grid img {{ max-width: {max_width}px; border-radius: 4px; }}
        .gif-grid .missing {{ color: #999; font-style: italic; }}
    </style>
    <h3>{building}</h3>
    <table class="gif-grid">
    <tr><th></th>
    '''
    for w in WEATHERS:
        html += f'<th>{WEATHER_NAMES[w]}</th>'
    html += '</tr>'
    
    for policy in POLICIES:
        html += f'<tr><td class="policy-header">{POLICY_NAMES[policy]}</td>'
        for weather in WEATHERS:
            gif_path = RESULTS_DIR / building / weather / policy / "simulation.gif"
            if gif_path.exists():
                # Relative path - Jupyter serves files from notebook directory
                rel_path = f"{RESULTS_DIR}/{building}/{weather}/{policy}/simulation.gif"
                html += f'<td><img src="{rel_path}" title="{weather}/{policy}"></td>'
            else:
                html += '<td class="missing">pending</td>'
        html += '</tr>'
    html += '</table>'
    return html

display(HTML(create_gif_grid(BUILDING)))

## All Buildings Comparison

In [ ]:
all_data = []
for b in list_buildings():
    df = load_summaries(b)
    if len(df) > 0:
        df['building'] = b
        all_data.append(df)

if all_data:
    all_df = pd.concat(all_data)
    summary = all_df.groupby(['building', 'policy'])['total_reward'].mean().unstack()
    summary = summary.reindex(columns=[p for p in POLICIES if p in summary.columns])
    print("Average Reward by Building and Policy:")
    display(summary.style.format("{:.1f}").background_gradient(cmap='RdYlGn', axis=None))
else:
    print("No data yet.")

## View Single GIF (larger)

In [ ]:
def show_gif(building, weather, policy):
    gif_path = RESULTS_DIR / building / weather / policy / "simulation.gif"
    if not gif_path.exists():
        print(f"Not found: {gif_path}")
        return
    print(f"{building} / {WEATHER_NAMES[weather]} / {POLICY_NAMES[policy]}")
    display(Image(filename=str(gif_path), width=500))

# Available:
for w in WEATHERS:
    for p in POLICIES:
        if (RESULTS_DIR / BUILDING / w / p / "simulation.gif").exists():
            print(f"show_gif('{BUILDING}', '{w}', '{p}')")

In [ ]:
# Uncomment to view:
# show_gif(BUILDING, "hot_summer", "all_off")